In [27]:
import pandas as pd

In [28]:
# carregar arquivos:
caminho = r"C:\Users\fabio.varriano\OneDrive - Sicoob\01_2025\fabrica_de_limites\06.ChequeEspecial\Data\Sisbr\RelEnquadramento0.31328899001719834.csv"
df1 = pd.read_csv(caminho, sep=';', encoding='latin1')

In [29]:
# dados de conta corrente do db
caminho2 = r"C:\Users\fabio.varriano\OneDrive - Sicoob\01_2025\fabrica_de_limites\06.ChequeEspecial\Data\data_parquet\conta_corrente"
df2 = pd.read_parquet(caminho2)

In [30]:
# carregar dados do PA 
caminho3 = r"C:\Users\fabio.varriano\OneDrive - Sicoob\01_2025\fabrica_de_limites\06.ChequeEspecial\Data\data_parquet\dados_pa.parquet"
df3 = pd.read_parquet(caminho3)

# EDA Limites

In [31]:
limite = df1.copy()

In [32]:
# Separar as colunas de interesse:
limite = limite[['Data / Hora Enquadramento', 'PA', 'Tipo pessoa',
        'CPF /CNPJ', 'Nome Associado', 'Perfil Tomador',
        'Submodalidade', 'Limite Automático',
       'Utilizado Automático']]
# filtrar submodalidade cheqeue especial 
limite = limite[limite['Submodalidade'] == 'CHEQUE ESPECIAL']
# filtrar Limite automático <> de zero e Utilizado automático == de zero
limite = limite[(limite['Limite Automático'] != '0,00') &
                (limite['Utilizado Automático'] == '0,00')]
# ajustar nome da coluna cpf-cnpj
limite.rename(columns={'CPF /CNPJ': 'CPF_CNPJ'}, inplace=True)
# excluir '.', '/' e '-' do cpf-cnpj
limite['CPF_CNPJ'] = limite['CPF_CNPJ'].str.replace('.', '')
limite['CPF_CNPJ'] = limite['CPF_CNPJ'].str.replace('/', '')
limite['CPF_CNPJ'] = limite['CPF_CNPJ'].str.replace('-', '')


# EDA Conta Corrente

In [33]:
# cópia do df2
cc = df2.copy()

In [ ]:
# filtar somente conta corrente ativa 
cc = cc[cc['Situacao_Conta_Corrente'] == 'ATIVA']
# Separar as colunas de interesse:
cc = cc[['CPF_CNPJ', 'Nome_Gerente',
       'Indicador_Cheque_Especial_Conta_Garantida',
       'Valor_Limite_Credito_Contratato']]
# agrupar CPF_CNPJ com valor de limite de crédito disponível para contratação 
cc = cc.groupby('CPF_CNPJ', ).agg({'Valor_Limite_Credito_Contratato': 'sum'}).reset_index()

In [42]:
# criar lista de cooperados e o nome do gerente 
gerente = df2.copy()
gerente = gerente[['CPF_CNPJ', 'Nome_Gerente']]
gerente.drop_duplicates(inplace=True)

In [43]:
gerente.head()

,CPF_CNPJ,Nome_Gerente
0,39183024824,LUCAS MOREIRA LOPES
1,43538035000191,ANDERSON RODRIGUES DE OLIVEIRA
2,70292436173,DANIEL SILVA COELHO
3,04400675106,ANA LUIZA SOARES SILVA
4,00435861123,VALDEY BARBOZA PASSOS


# Dados do PA 

In [35]:
pa = df3.copy()

In [36]:
# separar as colunas de interesse
pa = pa[['Regiões', 'GA Regional', 'Regional', 'Número PA', 'Nome PA']]
# alterar nome da columa PA 
pa.rename(columns={'Número PA': 'PA'}, inplace=True)

# Merge dfs


In [44]:
# Merge do df limites com o df cc
df = pd.merge(limite, cc, on='CPF_CNPJ', how='left')
# merge df com o df pa
df = pd.merge(df, pa, on='PA', how='left')
# slice excluir limite contratado <> 0
df = df[df['Valor_Limite_Credito_Contratato'] == 0]
# slice excluir valores nulos
df = df.dropna(subset=['Valor_Limite_Credito_Contratato'])
df = df.merge(gerente, on='CPF_CNPJ', how='left')

In [48]:
# reorganizar as colunas do df 
df = df[['Regiões', 'GA Regional', 'Nome PA','Nome_Gerente', 'CPF_CNPJ', 'Nome Associado','Perfil Tomador' , 'Limite Automático']]

In [49]:
df.columns

Index(['Regiões', 'GA Regional', 'Nome PA', 'Nome_Gerente', 'CPF_CNPJ',
       'Nome Associado', 'Perfil Tomador', 'Limite Automático'],
      dtype='object')

In [50]:
df.head()

,Regiões,GA Regional,Nome PA,Nome_Gerente,CPF_CNPJ,Nome Associado,Perfil Tomador,Limite Automático
0,SP2,SP2 - GR1 - Marcelino,108 - MARÍLIA,MARILIA GASPAR MAGNANE,08813745800,LUIS FERNANDO MARTINS,PERFIL PF 3,"10.000,00"
1,TO,TO - GR1 - Fábio Brito,13 - ARAGUAÍNA,GERSON BARBOSA PONTES,36443886000195,K. A. D. SILVA LTDA,PERFIL PJ 8,"2.500,00"
2,DIGITAL,DIGITAL,97 - ATENDIMENTO DIGITAL,DANILO ANDRADE MORAIS,92500595153,RODRIGO VIEIRA URANI,PERFIL PF-DIGITAL 2,"7.000,00"
3,DF,DF - GR1 - Valeria,09 - TAGUATINGA,JEFFERSON EDUARDO GOMES DA SILVA,00292734123,LAURENCE FURTADO E SOUZA,PERFIL PF-NOVO 1,"10.000,00"
4,DIGITAL,DIGITAL,97 - ATENDIMENTO DIGITAL,ANA LUIZA SOARES SILVA,79824994149,JANAINA BORGES DOS SANTOS SOUSA,PERFIL PF.DIGITAL-NOVO 3,"7.000,00"


In [51]:
# salvar arquivo 
df.to_excel(r'C:\Users\fabio.varriano\OneDrive - Sicoob\01_2025\fabrica_de_limites\06.ChequeEspecial\output\limite_cheque_especial_fabrica.xlsx', index=False)
